In [1]:
import numpy as np
from Prufer import Prufer_to_Tree
from Prufer import Tree_to_Prufer

In [2]:
# Function that builds the tree, The structure is a dictionary with entries in the form of "(node,node) : [(distribution, parameters), mgf, edge delay]"
#TODO Scikit-Bio tree class
#TODO Prufer sequence to my text file and inverse
#TODO look around at tree visualizers: different colored sources and observers, different edge lengths based on edge delays, ete3 (etetoolkit)
def Build_Tree(file_name):
    with open(file_name,"r") as filestream:

        tree = {}

        for line in filestream:
            curr = line.split(",")
            curr[len(curr)-1] = curr[len(curr)-1].replace('\n','')
            dist = curr[2]
            if dist == 'N':
                mu = float(curr[3])
                sigma2 = float(curr[4])
                tree[frozenset({curr[0],curr[1]})] = [(dist,mu,sigma2),lambda t:np.exp(mu*t)*np.exp((1/2)*(sigma2**2)*(t**2)),0]
            if dist == 'E':
                lam = float(curr[3])
                tree[frozenset({curr[0],curr[1]})] = [(dist,lam),lambda t: lam/(lam-t),0]
            if dist == 'U':
                a = float(curr[3])
                b = float(curr[4])
                tree[frozenset({curr[0],curr[1]})] = [(dist,a,b),lambda t: (np.exp(t*b)-np.exp(t*a))/(t*(b-a)),0]
            if dist == 'P':
                lam = float(curr[3])
                tree[frozenset({curr[0],curr[1]})] = [(dist,lam),lambda t: np.exp(lam*(np.exp(t)-1)),0]
        return tree

In [3]:
#Simulates the edge delay of a single edge
def Simulate(value):
    dist = value[0][0]
    if dist == 'N':
        value[2] = np.random.normal(value[0][1],value[0][2])
    if dist == 'E':
        value[2] = np.random.exponential(value[0][1])
    if dist == 'U':
        value[2] = np.random.uniform(value[0][1],value[0][2])
    if dist == 'P':
        value[2] = np.random.poisson(value[0][1])

#Uses the distribution and parameters stored in the tree to simulate the edge delays for each edge.
def Simulate_Edges(tree):
    for edge in tree:
        Simulate(tree[edge])

In [4]:
#Creates a tree object that can be more easily handled in the form of a dictionary where every key is a node and every value stored at the key is every adjacent node to the key
def Connection_Tree(tree):
    connection_tree = {}
    for keys in tree:

        nodes = []
        for node in keys:
            nodes.append(node)
        
        if nodes[0] not in connection_tree:
            connection_tree[nodes[0]] = set(nodes[1])
        if nodes[1] not in connection_tree:
            connection_tree[nodes[1]] = set(nodes[0])
        if nodes[0] in connection_tree:
            connection_tree[nodes[0]].add(nodes[1])
        if nodes[1] in connection_tree:
            connection_tree[nodes[1]].add(nodes[0])
    return connection_tree

#Does a recursive DFS on a tree between two given nodes
def DFS(connection_tree, source, observer):
    stack = [(source, [source])]
    visited = set()
    while stack:
        (vertex, path) = stack.pop()
        if vertex not in visited:
            if vertex == observer:
                return path
            visited.add(vertex)
            for neighbor in connection_tree[vertex]:
                stack.append((neighbor,path + [neighbor]))

#Converts a path in the form of consecutive nodes into a path in the form of consecutive edges
def Path_Edge(path):
    edges = []
    for i in range(len(path)-1):
        edges.append(frozenset({path[i],path[i+1]}))
    return edges

#Simulates the infection from a given source node to an observer node
def Infection_Simulation(tree, observers, source):
    infection_times = {}
    connection_tree = Connection_Tree(tree)

    #Finds the path each observer to the source using DFS and then adds up the edge costs stored in the tree on those paths
    for observer in observers:
        path = DFS(connection_tree, source, observer)
        edges = Path_Edge(path)
        time = 0
        for edge in edges:
            time += tree[edge][2]
        infection_times[observer] = time
    return infection_times

In [5]:
#Finds the joint moment generating function for a set of observer nodes with corresponding times and returns the mgf evaluated at those times.
#Returns a dictionary of the form "source : phi(t|s = source)"

def Joint_MGF(tree,connection_tree,observers):
    sources = {}

    #Finds the list of possible observer nodes
    for node in connection_tree:
        if node not in observers:
            sources[node] = 0

    #Finds the joint mgf for each possible source
    for node in sources:
        edges = []
        mgf = 1

        #creates an list where each element is the path from the source node to each observer
        for key in observers:
            path = DFS(connection_tree,node,key)
            edges.append([key,Path_Edge(path)])
        path_eval = {}

        #Creates a dictionary of the form "edge: sum of t's that use that edge"
        for i in range(len(edges)):
            for edge in edges[i][1]:
                if edge not in path_eval:
                    path_eval[edge] = observers[edges[i][0]]
                else:
                    path_eval[edge] += observers[edges[i][0]]

        #Evaluates the joint mgf using the dictionary
        for edge in path_eval:
            phi = tree[edge][1](path_eval[edge])
            mgf = mgf * phi
            
        sources[node] = mgf
    return sources

In [6]:
#Some Testing for the joint mgf function
Tree = Build_Tree("test.txt")
connection_tree = Connection_Tree(Tree)
Simulate_Edges(Tree)

ta = .4
td = .8

observers = {'a': ta, 'd':td}

print(Joint_MGF(Tree, connection_tree, observers))


print(Tree[frozenset({'a','b'})][1](ta)*Tree[frozenset({'c','b'})][1](td)*Tree[frozenset({'c','d'})][1](td)) #phi(t|s=b)

print(Tree[frozenset({'a','b'})][1](ta)*Tree[frozenset({'c','b'})][1](ta)*Tree[frozenset({'c','d'})][1](td)) #phi(t|s=c)

print(Tree[frozenset({'a','b'})][1](ta)*Tree[frozenset({'c','b'})][1](ta)*Tree[frozenset({'c','e'})][1](ta+td)*Tree[frozenset({'c','d'})][1](td)) #phi(t|s=e)

print(Tree[frozenset({'a','b'})][1](ta)*Tree[frozenset({'c','b'})][1](ta)*Tree[frozenset({'c','f'})][1](ta+td)*Tree[frozenset({'c','d'})][1](td)) #phi(t|s=e)

{'b': 7.817513617382336, 'c': 6.149474015700122, 'e': 11.889582274795437, 'f': 12.633683645845965}
7.817513617382336
6.149474015700122
11.889582274795437
12.633683645845963


In [9]:
tree = Prufer_to_Tree([4,2,6,7,2])
print(tree)
seq = Tree_to_Prufer(tree)
print(seq)

{1: [4], 2: [3, 6, 7], 3: [2], 4: [1, 6], 5: [7], 6: [4, 2], 7: [5, 2]}
[4, 2, 6, 7, 2]
